# TS4NLE: a Template System for Natural Language Explanations
TS4NLE converts the explanation of an eXplainable AI (XAI) system into natural language utterances comprehensible by humans. Moreover, the XAI explanation can be enhanced with a domain and a user model that allow the natural language rendering to be tailored to the particular user.

TS4NLE has been tested with real users for providing persuasive explanations for the adherance to the Mediterranean diet, see the main paper [here](https://www.sciencedirect.com/science/article/pii/S0933365719310140) and the [SEMEX4ALL tutorial](https://horus-ai.fbk.eu/semex4all/) at [ISWC2020](https://iswc2020.semanticweb.org/program/tutorials/).


## Installation
The source code of TS4NLE can be retrieved [here](https://github.com/ivanDonadello/TS4NLE). It requires Node.js and Python languages and it is mainly based on [RosaeNLG](https://rosaenlg.org/): a Natural Language Generation library for node.js, based on the [Pug template engine](https://pugjs.org/). Once Node.js, Python and the nmp package manager have been installed, TS4NLE can be easily installed by installing, as first step, the [Graphviz](https://graphviz.org/) library for graph visualization:

In [ ]:
!apt-get install graphviz graphviz-dev
!pip install pygraphviz

then install the repository with:

In [ ]:
!git clone https://github.com/radu1690/Explanations.git
%cd Explanations
!npm install

Remove the initial characters `!`, `%` when running in a shell.


## Citing TS4NLE
If you use TS4NLE in your research, please use the following BibTeX entry.
```
@article{DragoniDE20,
  author    = {Mauro Dragoni and
               Ivan Donadello and
               Claudio Eccher},
  title     = {Explainable {AI} meets persuasiveness: Translating reasoning results into behavioral change advice},
  journal   = {Artif. Intell. Medicine},
  volume    = {105},
  pages     = {101840},
  year      = {2020}
}
```


## Using TS4NLE
TS4NLE requires as input:
- An explanation graph provided by XAI techniques that contains all the information necessary to provide a justification for the output of an Artificial Intelligence system. Notice that TS4NLE is agnostic to the underlying AI system (no matter what whether it is a Machine-Learning based or a reasoning-based one) as it is solely based on the explanation graph.
- The templates that encode the rules on how the explanation graph has to be rendered in natural language. They can be intepreted as a bridge between the explanation graph and the consequent natural language explanation. They both contain chunks of text and the link/rules to the nodes and relations in the explanation graph.

The following sections contain howtos about the encoding of explanation graphs and of the templates.


## How to encode the explanation graphs for TS4NLE
The graphs are stored in `.csv` files (with tab delimiter) where each row contains a triple in the form of `<subject, predicate, object>`: `subject` is the parent node, `object` is the child node and `predicate` is the edge. Every XAI method is responsible to provide the right semantics to the triple `<subject, predicate, object>`. For example, the bars provided by the Lime explanator can be encoded with nodes having connections with the root (the output) and with their children that encode the numeric values associated with the features.

According to the current implementation, the explanation graph must be in the form of a tree: a direct acyclic graph where children nodes have only one parent. The subject of the first row in the `.csv` file is the root of the explanation graph. Here a small example of an explanation graph detecting a bad diet behavior:
```
explanation outputClassification badBehavior 
explanation	entity	RedMeat
explanation	timing	Week
RedMeat	quantity	7
RedMeat	Nutrient	AnimalLipid
AnimalLipid	cons	increment of cardiovascular disease risk
AnimalLipid	cons	increment of cholesterol in the blood
explanation hasUser Mario
Mario hasAge 65
Mario hasRecidivityFor RedMeat
RedMeat recidivityLevel high
```
The root of the graph is the concept of `explanation` linked with the output of our classifier: `badBehavior` (first line). What follows is the explanation for the detected bad behavior (lines 2-7): a too much consumption of red meat in only one week. Lines 5-7 contain information from a domain model stating what are the medium-term consequences of a high consumption of red meat. Lines 8-10 contain the user model, that is, information about the user diet habits. This can be used to tailor the natural language rendering of the explanation.

The following commad will show a graphical representation of the explanation graph:
```
python csv2graphviz.py <graph.csv>
```

### Parsing the explanation graph
The script `graphToObject.js` will parse the `.csv` file into an `explanation` Javascript object that can be inspected in the Pug template, see the next section for more details. Such a script can be embedded into a main file with the following code:
```
graph2Object.main(dir_explanationGraph)
.then(data =>{
    var output = rosaenlgPug.renderFile(dir_template, {
        language: rosae_language,
        explanation: data,
        pluralize: pluralize,
        random: randomElement
    });
    console.log(output)
})
.catch(err => console.error(err))
```
According to the current implementation, the fields of the `explanation` object can be only single-value fields or arrays.


## How to write your own TS4NLE templates
The TS4NLE templates follow the Pug syntax (see the main website [here](https://pugjs.org/) for more information) and are rendered with RosaeNLG engine. Here we describe the basics of Pug and RosaeNLG, for a complete tutorial with advanced features see the main website [here](https://rosaenlg.org/rosaenlg/3.0.0/tutorials/tutorial_en_US.html).


### Rendering plain text
Templates are textual files in `.pug` format. To just render some plain text use the pipe character (`|`) followed by the text you want to render:  
```
|Pay attention at your diet!
```
with output: "Pay attention at your diet!". Spaces between different plain texts (except only punctuation) are automatically inserted, so:
```
|Pay attention
|at your diet
|!
```
will still output: "Pay attention at your diet!".

### Rendering explanation fields
Once the explanation graph has been parsed and transformed into an `explanation` Javascript object, it is possible to access its fields in the Pug template. The access to the single fields can be done in two ways:
 - `#[+value(explanation.field)]` that throws an error if the field does not exists;
 - `!{explanation.field}` that prints an empty string if the field does not exists.

For example, rendering the English label of a food entity is simply performed by accessing the relative field:  
```
|You ate #[+value(explanation.entity.enLabel)].
```
with output: "You ate red meat."


### Rendering conditionals
If-then-else statements are allowed in the templates and can encode some rules of rendering according to the content of the explanation graph. For example, one can select the more appropriate textual content according to the kind of food he has consumed:
```
if explanation.constraint == 'less'
	|You ate too much
else 
	|You did not eat enough
|#[+value(explanation.entity.enLabel)].
```  
with output: "You ate too much red meat." if `explanation.constraint` is `less` and `explanation.entity.enLabel` is `red meat`. Other values for these fields can lead to the following output: "You did not eat enough vegetables." Notice that the use of the `#[+value()]` or the `!{}` notations is not allowed inside the `if`, `else` conditions.  

### Redering Javascript code
The Pug templates allow the writing of Javascript code. Here we just show the definition of simple variables that can be used in the template. Variables are defined with the following syntax:
```
- let variable_name = "variable value"
```
and accessed by `!{variable_name}`. The following template
```
- let food = "Red meat"
- let nutrient = ["animal proteins", "animal lipids"]

|!{food} contains a lot of !{nutrient[0]}
```
will output "Red meat contains a lot of animal proteins".


### Rendering loop iterations
For loops are useful when inspecting an array field in the explanation. Loops are performed with the `eachz` operator from RosaeNLG:
```
-let nutrients = ["animal lipids", "animal proteins"]
eachz element in nutrients
    |explanation.entity !{element}
```
with output: "Red meat contains animal lipids, animal proteins". However, this sentence is not formatted in a proper English. It should have an "and" instead of the comma and should end with a dot. This can be correct by using [additional parameters](https://rosaenlg.org/rosaenlg/3.0.0/mixins_ref/eachz_itemz.html) supported by the `eachz` operator. The previous template can be modified as this:
```
-let nutrients = ["animal lipids", "animal proteins"]
eachz element in nutrients with {separator: ",", last_separator: "and", end: "."} 
    |explanation.entity !{element}
```
with output: "Red meat contains animal lipids and animal proteins.".


## Functions
Javascript functions can be directly used in your templates. Here we provide two examples: the first involves a function of the standard Javascript library, the second a custom function.

### Pluralize
[Pluralize](https://www.npmjs.com/package/pluralize) is Javascript module that pluralize and singularize any given word. Use `pluralize(<word>)` or `pluralize.singular(<word>)` in a template to either pluralize or singularize a word. Use `pluralize.isSingular()` and `pluralize.isPlural()` to check if a word is either singular or plural. This example:  
```
if pluralize.isSingular(explanation.entity)
	|contains
else
	|contain
```  
will output "contains" if the entity field is singular (e.g., "red meat") or "contain" if the entity field is plural (e.g., "vegetables").


### Random
Random is a custom function that returns a random element of an array (if the argument is an array) or the argument itself if it is not an array. Here how to call the function in a template:  
```
- let nutrient = random(explanation.entity.negative)
- let consequence = random(nutrient.cons_en)

|High consumption of !{explanation.entity} for long time could cause !{consequence}.
```
with output "High consumption of red meat could cause cardiovascular diseases". Here the implementation of such a function in the `explanations.js` file:
```
function randomElement(items){
    if(Array.isArray(items)){
        return items[Math.floor(Math.random()*items.length)]
    }else{
        return items
    }
}
```


### How to add a function
Functions need to be passed to the RosaeNLG engine in your main file (`explanations.js` in our case):
```
graph2Object.main(dir_explanationGraph)
	.then(data =>{
		var output = rosaenlgPug.renderFile(dir_template, {
			language: rosae_language,
			explanation: data,
			pluralize: pluralize,	//pluralize function
			random: randomElement	//random function
		});
		console.log(output)
	})
```

## Run TS4NLE
You can run TS4NLE with:  
```
node explanations.js <template> <graph>
```
where `template` is the `.pug` file of the template and `graph` the `.csv` explanation graph file. Here some examples in the repository:

In [ ]:
node explanations.js templates/english/argument.pug graphs/exp_graph_1_en.csv

Red meat contains animal protein which can cause the increment of cancer risk!


You can specify the language, English or Italian (the default is English), like in the following example:

In [ ]:
!node explanations.js templates/italian/feedback.pug graphs/exp_graph_1_it.csv italian

Hai consumato una grande quantità di carne rossa (3 su un massimo di 2)
